In [ ]:
# 获取zero-shot的结果
from utils import save_data

import json


def package_sqls(
    sql_path, db_root_path, engine, sql_dialect="SQLite", mode="gpt", data_mode="dev"
):
    clean_sqls = []
    db_path_list = []
    if mode == "gpt":
        # use chain of thought
        sql_data = json.load(path)
        for _, sql_str in sql_data.items():
            if type(sql_str) == str:
                sql, db_name = sql_str.split("\t----- bird -----\t")
            else:
                sql, db_name = " ", "financial"
            clean_sqls.append(sql)
            db_path_list.append(db_root_path + db_name + "/" + db_name + ".sqlite")


In [ ]:

def load_json(dir):
    with open(dir, "r") as j:
        contents = json.loads(j.read())
    return contents

def compute_acc_by_diff(exec_results, diff_json_path, results_path, metric="EX"):
    num_queries = len(exec_results)
    contents = load_json(diff_json_path)
    simple_results, moderate_results, challenging_results = [], [], []
    
    batch_size = 10
    for i, content in enumerate(contents):
        if content["difficulty"] == "simple":
            simple_results.append(exec_results[i])

        if content["difficulty"] == "moderate":
            moderate_results.append(exec_results[i])

        if content["difficulty"] == "challenging":
            try:
                challenging_results.append(exec_results[i])
            except:
                print(i)

        # Process every batch_size examples
        if (i + 1) % batch_size == 0 or i + 1 == num_queries:
            simple_acc = sum([res["res"] for res in simple_results]) / len(simple_results) if simple_results else 0
            moderate_acc = sum([res["res"] for res in moderate_results]) / len(moderate_results) if moderate_results else 0
            challenging_acc = sum([res["res"] for res in challenging_results]) / len(challenging_results) if challenging_results else 0
            all_acc = sum([res["res"] for res in exec_results[:i+1]]) / (i + 1)

            count_lists = [
                len(simple_results),
                len(moderate_results),
                len(challenging_results),
                i + 1,
            ]

            score_lists = [
                simple_acc * 100,
                moderate_acc * 100,
                challenging_acc * 100,
                all_acc * 100,
            ]

            save_data(i + 1, results_path, score_lists, count_lists, metric)

def save_data(idx, results_path, score_lists, count_lists, metric):
    levels = ["simple", "moderate", "challenging", "total"]
    
    with open(results_path, 'a') as file:
        file.write(f"Result Index: {idx}\n")
        file.write("{:20} {:20} {:20} {:20} {:20}\n".format("", *levels))
        file.write("{:20} {:<20} {:<20} {:<20} {:<20}\n".format("count", *count_lists))

        file.write(
            f"======================================    {metric}    =====================================\n"
        )

        formatted_scores = [
            f"{score:.2f}" if score is not None else "N/A" for score in score_lists
        ]
        file.write("{:20} {:<20} {:<20} {:<20} {:<20}\n".format(metric, *formatted_scores))


In [4]:
mistake_examples = []

mistake_prompt = '\n\n'.join(
    [
        f"example{index+1}: {{\n" + 
        '\n'.join([f"{key}: {value}" for key, value in example.items() if key != 'difficulty']) + 
        "\n}"
        for index, example in enumerate(mistake_examples)
    ]
)
assert mistake_prompt!=""

AssertionError: 

In [2]:
predicted_sql = ""
assert predicted_sql != ""

AssertionError: 

In [1]:
from openai import OpenAI
client = OpenAI(
                base_url="http://localhost:28083/v1/",
                api_key="Empty",
            )
instruction = "Hi. "
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "{0}：{1}".format(instruction,input)},
]
       
completion = client.chat.completions.create(
    model="Qwen2-57B-A14B-Instruct-GPTQ-Int4",
    messages=messages,
    stream=False, #设置是否流式输出
    #max_tokens=1000,
    top_p=1
)
print(completion.choices[0].message.content)